In [1]:
import sqlite3
import os
from typing import List
import re
import sqlite3
import sys

In [2]:
def join_broken_lines(sql_raw: str) -> str:
    # Join lines ending mid-word (like INTEG\nER(...)) with the next line
    return re.sub(r'(\w+)\n(\w+)', r'\1\2', sql_raw)


In [3]:
def convert_mysql_to_sqlite_syntax(mysql_sql_content: str) -> str:
    sql = mysql_sql_content

    sql = sql.replace('`', '')  # Remove backticks

    sql = re.sub(r'\bINTEG\s*\n\s*ER\b', 'INTEGER', sql, flags=re.IGNORECASE)
    sql = re.sub(r'\bCHARAC\s*\n\s*TER\b', 'CHARACTER', sql, flags=re.IGNORECASE)

    # Replace types
    sql = re.sub(r'\bINTEGER\(\d+\)', 'INTEGER', sql, flags=re.IGNORECASE)
    sql = re.sub(r'\bTINYINT(?:\(\d+\))?\b', 'INTEGER', sql, flags=re.IGNORECASE)
    sql = re.sub(r'\bINT(?:\(\d+\))?\b', 'INTEGER', sql, flags=re.IGNORECASE)
    sql = re.sub(r'\bVARCHAR(?:\(\d+\))?\b', 'TEXT', sql, flags=re.IGNORECASE)
    sql = re.sub(r'\bCHAR(?:\(\d+\))?\b', 'TEXT', sql, flags=re.IGNORECASE)
    sql = re.sub(r'\bTEXT\(\d+\)', 'TEXT', sql, flags=re.IGNORECASE)
    sql = re.sub(r'\bLONGTEXT\b', 'TEXT', sql, flags=re.IGNORECASE)
    sql = re.sub(r'\bMEDIUMTEXT\b', 'TEXT', sql, flags=re.IGNORECASE)
    sql = re.sub(r'\bBLOB(?:\(\d+\))?\b', 'BLOB', sql, flags=re.IGNORECASE)
    sql = re.sub(r'\bDATETIME(?:\(\d+\))?\b', 'TEXT', sql, flags=re.IGNORECASE)
    sql = re.sub(r'\bDATE(?:\(\d+\))?\b', 'TEXT', sql, flags=re.IGNORECASE)

    # Remove problematic SQL features
    sql = re.sub(r'\bAUTO_INCREMENT\b', '', sql, flags=re.IGNORECASE)
    sql = re.sub(r'\bUNSIGNED\b', '', sql, flags=re.IGNORECASE)
    sql = re.sub(r'DEFAULT\s+NULL', '', sql, flags=re.IGNORECASE)
    sql = re.sub(r'DEFAULT\s+\'null\'', '', sql, flags=re.IGNORECASE)
    sql = re.sub(r'DEFAULT\s+\'(\d+)\'', r'DEFAULT \1', sql, flags=re.IGNORECASE)

    # Fix TEXT column named 'TEXT'
    sql = re.sub(r'\bTEXT\s+TEXT\b', '"TEXT" TEXT', sql, flags=re.IGNORECASE)

    # Replace default CURRENT_TIMESTAMP
    sql = re.sub(r'DEFAULT\s+CURRENT_TIMESTAMP(?:\(\))?', "DEFAULT (datetime('now'))", sql, flags=re.IGNORECASE)

    # Remove table options
    sql = re.sub(r'\s+ENGINE=\w+\b', '', sql, flags=re.IGNORECASE)
    sql = re.sub(r'\s+DEFAULT\s+CHARSET=\w+\b', '', sql, flags=re.IGNORECASE)
    sql = re.sub(r'\s+COLLATE\s+\w+\b', '', sql, flags=re.IGNORECASE)

    # Remove KEYs/INDEXes
    sql = re.sub(r'^\s*(UNIQUE\s+)?KEY\s+\w+\s*\([^)]+\)\s*,?\s*$', '', sql, flags=re.MULTILINE | re.IGNORECASE)
    sql = re.sub(r'^\s*(INDEX|KEY)\s+\w+\s*\([^)]+\)\s*,?\s*$', '', sql, flags=re.MULTILINE | re.IGNORECASE)

    # Remove foreign key constraints
    sql = re.sub(r'CONSTRAINT\s+\w+\s+FOREIGN\s+KEY\s*\([^)]+\)\s+REFERENCES\s+[^\n,]+(?:,\n|\n|,)?', '', sql, flags=re.IGNORECASE)

    # Remove other MySQL stuff
    sql = re.sub(r'DELIMITER\s+\S+', '', sql, flags=re.IGNORECASE)
    sql = re.sub(r'LOCK TABLES\s+[^\;]+;', '', sql, flags=re.IGNORECASE)
    sql = re.sub(r'UNLOCK TABLES\s*;', '', sql, flags=re.IGNORECASE)
    sql = re.sub(r'/\*\![0-9]+.*?\*/', '', sql, flags=re.DOTALL)
    sql = re.sub(r'^\s*SET\s+[^;]+;', '', sql, flags=re.MULTILINE | re.IGNORECASE)
    sql = re.sub(r'SET\s+[^;]+;', '', sql, flags=re.IGNORECASE)
    sql = re.sub(r'^=\s*[^;]+;', '', sql, flags=re.MULTILINE)

    # Clean up excessive semicolons
    sql = re.sub(r'^\s*;\s*$', '', sql, flags=re.MULTILINE)  # kill lines that are just ;
    sql = re.sub(r';\s*;+', ';', sql)  # collapse multiple ;; into a single ;

    sql = re.sub(r'(\n\s*\w+\s*)\n\s*', r'\1 ', sql)  # join identifier line with type line
    sql = re.sub(r'^\s*\w+\s*(,)?\s*$', '', sql, flags=re.MULTILINE)  

    # Final cleanup
    sql = re.sub(r',\s*\)', ')', sql)  # trailing commas
    sql = re.sub(r',\s*,', ',', sql)
    sql = '\n'.join(line for line in sql.splitlines() if line.strip())  # remove blank lines

    sql = re.sub(r'--.*MySQL dump.*', '', sql, flags=re.IGNORECASE)

    return sql.strip()


In [4]:
def process_sql_files_for_sqlite(input_folder, output_folder, sql_file_names):
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)
        print(f"Created output folder for converted SQL files: {output_folder}")

    converted_paths = []
    for file_name in sql_file_names:
        input_file_path = os.path.join(input_folder, file_name)
        output_file_name = file_name.replace('.sql', '_sqlite.sql')
        output_file_path = os.path.join(output_folder, output_file_name)

        if not os.path.exists(input_file_path):
            print(f"Skipping: Input file not found: {input_file_path}")
            continue

        print(f"Converting '{file_name}' to SQLite syntax...")
        try:
            with open(input_file_path, 'r', encoding='utf-8') as f_in:
                mysql_content = f_in.read()

            mysql_content = join_broken_lines(mysql_content)
            sqlite_content = convert_mysql_to_sqlite_syntax(mysql_content)

            with open(output_file_path, 'w', encoding='utf-8') as f_out:
                f_out.write(sqlite_content)
            print(f"Successfully converted and saved to: {output_file_path}")
            converted_paths.append(output_file_path)
        except Exception as e:
            print(f"Error converting '{file_name}': {e}")
            import traceback
            traceback.print_exc() # Print full error for debugging

    return converted_paths

In [5]:
def initialise_db(db_uri: str, sql_files: List[str], overwrite_existing: bool = False):
    db_file_path = db_uri.replace("sqlite:///", "")

    if overwrite_existing and os.path.exists(db_file_path):
        print(f"Database file '{db_file_path}' exists. 'overwrite_existing' is True. Deleting it...")
        try:
            os.remove(db_file_path)
            print(f"Existing database file '{db_file_path}' deleted successfully.")
        except OSError as e:
            print(f"Error deleting existing database file '{db_file_path}': {e}")
            raise

    db_dir = os.path.dirname(db_file_path)
    if db_dir and not os.path.exists(db_dir):
        os.makedirs(db_dir, exist_ok=True)
        print(f"Created directory for database: {db_dir}")

    conn = None
    try:
        conn = sqlite3.connect(db_file_path)
        cursor = conn.cursor()
        print(f"Connected to database file: {db_file_path}")

        for script_path in sql_files:
            if not os.path.exists(script_path):
                print(f"Warning: SQL population script not found: {script_path}. Skipping.")
                continue

            with open(script_path, 'r', encoding='utf-8', errors='ignore') as f:
                raw_sql = f.read()

            converted_sql = convert_mysql_to_sqlite_syntax(raw_sql)

            if "LongCastId" in converted_sql:
                print("--- RAW SQL CONTEXT ---")
                idx = converted_sql.index("LongCastId")
                print(converted_sql[max(0, idx - 100): idx + 200])

            print(f"Executing script: {script_path}")
            print(converted_sql[:500])
            print(f"----- START SQL DUMP for {script_path} -----")
            print(converted_sql[:1000])
            print(f"----- END SQL DUMP -----")
            cursor.executescript(converted_sql)
            print(f"Executed SQL script: {script_path}")

        conn.commit()
        print(f"Database '{db_file_path}' populated successfully from provided scripts.")

    except sqlite3.Error as e:
        print(f"SQLite error during database initialization: {e}")
        if conn:
            conn.rollback()
        raise
    except Exception as e:
        print(f"An unexpected error occurred during database initialization: {e}")
        raise
    finally:
        if conn:
            conn.close()
            print("Database connection closed.")

In [6]:
current_notebook_dir = os.path.dirname(os.path.abspath(''))

input_sql_folder = os.path.join(current_notebook_dir, 'OPMS_sql_files')
output_sqlite_sql_folder = os.path.join(current_notebook_dir, 'OPMS_sql_files_sqlite_converted') 


In [7]:
print(current_notebook_dir)

c:\Users\caio\code\maritime_report_generation


In [8]:
print(input_sql_folder)

c:\Users\caio\code\maritime_report_generation\OPMS_sql_files


In [9]:
print(output_sqlite_sql_folder)

c:\Users\caio\code\maritime_report_generation\OPMS_sql_files_sqlite_converted


In [10]:
absolute_db_path = os.path.join(current_notebook_dir, 'sql_files', 'myDataBase.db')
db_uri = f"sqlite:///{absolute_db_path.replace(os.sep, '/')}"

In [11]:
print(db_uri)

sqlite:///c:/Users/caio/code/maritime_report_generation/sql_files/myDataBase.db


In [12]:
your_original_sql_file_names = [
    'OPMS_ANC.sql',
    'OPMS_DNO.sql',
    'OPMS_ENC.sql',
    'OPMS_FNO.sql',
    'OPMS_SNC.sql',
    'OPMS_WNC.sql',
]

In [13]:
converted_sql_file_paths=process_sql_files_for_sqlite(
    input_sql_folder,
    output_sqlite_sql_folder, 
    your_original_sql_file_names
)

Converting 'OPMS_ANC.sql' to SQLite syntax...
Successfully converted and saved to: c:\Users\caio\code\maritime_report_generation\OPMS_sql_files_sqlite_converted\OPMS_ANC_sqlite.sql
Converting 'OPMS_DNO.sql' to SQLite syntax...
Successfully converted and saved to: c:\Users\caio\code\maritime_report_generation\OPMS_sql_files_sqlite_converted\OPMS_DNO_sqlite.sql
Converting 'OPMS_ENC.sql' to SQLite syntax...
Successfully converted and saved to: c:\Users\caio\code\maritime_report_generation\OPMS_sql_files_sqlite_converted\OPMS_ENC_sqlite.sql
Converting 'OPMS_FNO.sql' to SQLite syntax...
Successfully converted and saved to: c:\Users\caio\code\maritime_report_generation\OPMS_sql_files_sqlite_converted\OPMS_FNO_sqlite.sql
Converting 'OPMS_SNC.sql' to SQLite syntax...
Successfully converted and saved to: c:\Users\caio\code\maritime_report_generation\OPMS_sql_files_sqlite_converted\OPMS_SNC_sqlite.sql
Converting 'OPMS_WNC.sql' to SQLite syntax...
Successfully converted and saved to: c:\Users\cai

In [14]:
overwrite_existing_db = True 

In [15]:
initialise_db(db_uri, converted_sql_file_paths, overwrite_existing_db)

Database file 'c:/Users/caio/code/maritime_report_generation/sql_files/myDataBase.db' exists. 'overwrite_existing' is True. Deleting it...
Existing database file 'c:/Users/caio/code/maritime_report_generation/sql_files/myDataBase.db' deleted successfully.
Connected to database file: c:/Users/caio/code/maritime_report_generation/sql_files/myDataBase.db
--- RAW SQL CONTEXT ---
GER NOT NULL ,
  "TEXT" TEXT ,
  ActionName TEXT ,
  Status INTEGER DEFAULT 0,
  UserId INTEGER ,
  LongCastId INTEGER ,
  PRIMARY KEY (ActionPendingId));
--
-- Dumping data for table actionpending
--
--
-- Table structure for table aircraftallocated
--
DROP TABLE IF EXISTS aircraftallocated;
CREATE
Executing script: c:\Users\caio\code\maritime_report_generation\OPMS_sql_files_sqlite_converted\OPMS_ANC_sqlite.sql
--
-- Host: localhost    Database: OPMS_ANC
-- ------------------------------------------------------
-- Server version	10.1.18-MariaDB
--
-- Table structure for table actionpending
--
DROP TABLE IF EXISTS

OperationalError: near "=": syntax error